In [2]:
# Standard block to equalize local and Colab.
from pathlib import Path
import os
try:
    # See if we are running on google.colab
    import google.colab
    from google.colab import files
    # Configure kaggle
    files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
    !pip install -q kaggle
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    # Download the workshop repo and change to its directory
    # For now edit the username/password. This requirement will be removed when the repo is made public.
    !git clone https://<username>:<password>@github.com/SachsLab/IntracranialNeurophysDL.git ../repo
    os.chdir('../repo')
    IN_COLAB = True
except ModuleNotFoundError:
    import sys
    if Path.cwd().stem == 'notebooks':
        os.chdir(Path.cwd().parent)
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')
    IN_COLAB = False
    

In [3]:
# Download faces_basic if we don't already have it.
datadir = Path.cwd() / 'data' / 'kjm_ecog'
if not (datadir / 'converted').is_dir():
    !kaggle datasets download -d cboulay/kjm-ecog-faces-basic
    print("Finished downloading. Now extracting contents...")
    data_path = Path('kjm-ecog-faces-basic.zip')
    import zipfile
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        zip_ref.extractall(datadir / 'converted' / 'faces_basic')
    data_path.unlink()


In [4]:
# Load data from one participant.
SUB_ID = 'de'
from data.utils.fileio import from_neuropype_h5
test_file = datadir / 'converted' / 'faces_basic' / (SUB_ID + '_bp.h5')
chunks = from_neuropype_h5(test_file)
chunk_names = [_[0] for _ in chunks]
chunk = chunks[chunk_names.index('signals')][1]
ax_types = [_['type'] for _ in chunk['axes']]
instance_axis = chunk['axes'][ax_types.index('instance')]
n_trials = len(instance_axis['data'])
X = chunk['data'].reshape((n_trials, -1))  # 603 trials x 527 features
Y = instance_axis['data']['Marker'].values
n_trials, n_features = X.shape
print(X.shape, Y.shape)


(603, 527) (603,)



[Keras](), as provided by tensorflow

[tf.eager tutorial](https://adventuresinmachinelearning.com/tensorflow-eager-tutorial/)



In [13]:
import tensorflow as tf


# Enable eager execution so TensorFlow operations execute immediately and their result can be inspected.
# This is the default behaviour in upcoming TensorFlow 2.0 so we might as well get used to it.
tf.enable_eager_execution()

# Create a model that is a linear stack of layers. Well, only 1 layer. And a bad one at that.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(3, input_shape=(n_features,), activation='linear'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# See losses in tf.losses.
# See optimizer in tf.keras.optimizers: 'Adam', 'rmsprop', 'SGD'

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 3)                 1584      
Total params: 1,584
Trainable params: 1,584
Non-trainable params: 0
_________________________________________________________________


[tf.data](https://www.tensorflow.org/guide/datasets)

[tf.data tutorial](https://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/)



In [ ]:
BATCH_SIZE = 5

# Import our data as a Dataset.
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset = dataset.shuffle(1000).repeat().batch(BATCH_SIZE)
ds_iter = dataset.make_initializable_iterator()
next_element = ds_iter.get_next()
